<h1>Homework 04 - Applied ML</h1>

Importation of libraries

In [2]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import datetime
from sklearn.ensemble import RandomForestClassifier

Importation of the datas

In [3]:
filename = os.path.join('data','CrowdstormingDataJuly1st.csv') 
df = pd.read_csv(filename)
df.head()

,playerShort,player,club,leagueCountry,birthday,height,weight,position,games,victories,...,rater2,refNum,refCountry,Alpha_3,meanIAT,nIAT,seIAT,meanExp,nExp,seExp
0,lucas-wilchez,Lucas Wilchez,Real Zaragoza,Spain,31.08.1983,177.0,72.0,Attacking Midfielder,1,0,...,0.50,1,1,GRC,0.326391,712.0,0.000564,0.396000,750.0,0.002696
1,john-utaka,John Utaka,Montpellier HSC,France,08.01.1982,179.0,82.0,Right Winger,1,0,...,0.75,2,2,ZMB,0.203375,40.0,0.010875,-0.204082,49.0,0.061504
2,abdon-prats,Abdón Prats,RCD Mallorca,Spain,17.12.1992,181.0,79.0,NaN,1,0,...,NaN,3,3,ESP,0.369894,1785.0,0.000229,0.588297,1897.0,0.001002
3,pablo-mari,Pablo Marí,RCD Mallorca,Spain,31.08.1993,191.0,87.0,Center Back,1,1,...,NaN,3,3,ESP,0.369894,1785.0,0.000229,0.588297,1897.0,0.001002
4,ruben-pena,Rubén Peña,Real Valladolid,Spain,18.07.1991,172.0,70.0,Right Midfielder,1,1,...,NaN,3,3,ESP,0.369894,1785.0,0.000229,0.588297,1897.0,0.001002


<h2>Data exploration</h2>

In [4]:
print('Number of dyads: ', len(df))
print('Total number of interactions between a referee and a player: ', sum(df.games))

Number of dyads:  146028
Total number of interactions between a referee and a player:  426572


In [5]:
print('Mean number of games for a dyad: ', np.mean(df['games']))

Mean number of games for a dyad:  2.921165803818446


## Data set up

In [6]:
print("We have", df.size,"rows.")

We have 4088784 rows.


We decided to remove row where the two rates were significantly different or if any of the rates were absent (Nan value).

In [7]:
cleandf = pd.DataFrame()

In [8]:
cleandf = df[df["rater1"].notnull() & df["rater2"].notnull()]
cleandf['mean'] = abs(cleandf.rater1 - cleandf.rater2)

cleandf = cleandf[cleandf['mean'] <= 0.25]
cleandf.drop('mean', axis =1, inplace=True)

C:\Users\Sabrine\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [9]:
cleandf.head()

,playerShort,player,club,leagueCountry,birthday,height,weight,position,games,victories,...,rater2,refNum,refCountry,Alpha_3,meanIAT,nIAT,seIAT,meanExp,nExp,seExp
0,lucas-wilchez,Lucas Wilchez,Real Zaragoza,Spain,31.08.1983,177.0,72.0,Attacking Midfielder,1,0,...,0.50,1,1,GRC,0.326391,712.0,0.000564,0.396000,750.0,0.002696
1,john-utaka,John Utaka,Montpellier HSC,France,08.01.1982,179.0,82.0,Right Winger,1,0,...,0.75,2,2,ZMB,0.203375,40.0,0.010875,-0.204082,49.0,0.061504
5,aaron-hughes,Aaron Hughes,Fulham FC,England,08.11.1979,182.0,71.0,Center Back,1,0,...,0.00,4,4,LUX,0.325185,127.0,0.003297,0.538462,130.0,0.013752
6,aleksandar-kolarov,Aleksandar Kolarov,Manchester City,England,10.11.1985,187.0,80.0,Left Fullback,1,1,...,0.25,4,4,LUX,0.325185,127.0,0.003297,0.538462,130.0,0.013752
7,alexander-tettey,Alexander Tettey,Norwich City,England,04.04.1986,180.0,68.0,Defensive Midfielder,1,0,...,1.00,4,4,LUX,0.325185,127.0,0.003297,0.538462,130.0,0.013752


In [10]:
print("And now we have",cleandf.size,"rows")

And now we have 3484796 rows


In [11]:
print('Number of dyads: ', len(cleandf))
print('Total number of interactions between a referee and a player: ', sum(cleandf.games))

Number of dyads:  124457
Total number of interactions between a referee and a player:  372628


Then we decide to take the skin tone as the mean between the two rates

In [12]:
cleandf["meanSkinTone"] = abs(cleandf["rater1"] - cleandf["rater2"] ) / 2
cols = cleandf.columns.tolist()
cols = cols[-1:] + cols[:-1]
cleandf = cleandf[cols]
cleandf.head()

,meanSkinTone,playerShort,player,club,leagueCountry,birthday,height,weight,position,games,...,rater2,refNum,refCountry,Alpha_3,meanIAT,nIAT,seIAT,meanExp,nExp,seExp
0,0.125,lucas-wilchez,Lucas Wilchez,Real Zaragoza,Spain,31.08.1983,177.0,72.0,Attacking Midfielder,1,...,0.50,1,1,GRC,0.326391,712.0,0.000564,0.396000,750.0,0.002696
1,0.000,john-utaka,John Utaka,Montpellier HSC,France,08.01.1982,179.0,82.0,Right Winger,1,...,0.75,2,2,ZMB,0.203375,40.0,0.010875,-0.204082,49.0,0.061504
5,0.125,aaron-hughes,Aaron Hughes,Fulham FC,England,08.11.1979,182.0,71.0,Center Back,1,...,0.00,4,4,LUX,0.325185,127.0,0.003297,0.538462,130.0,0.013752
6,0.125,aleksandar-kolarov,Aleksandar Kolarov,Manchester City,England,10.11.1985,187.0,80.0,Left Fullback,1,...,0.25,4,4,LUX,0.325185,127.0,0.003297,0.538462,130.0,0.013752
7,0.000,alexander-tettey,Alexander Tettey,Norwich City,England,04.04.1986,180.0,68.0,Defensive Midfielder,1,...,1.00,4,4,LUX,0.325185,127.0,0.003297,0.538462,130.0,0.013752


In [13]:
def time_to_seconds(t):
    seconds = (pd.to_datetime(t) - datetime.datetime(1970, 1,1)).total_seconds()
    return int(seconds)

cleandf.birthday = cleandf.birthday.apply(time_to_seconds)

In [14]:
cleandf.head()

,meanSkinTone,playerShort,player,club,leagueCountry,birthday,height,weight,position,games,...,rater2,refNum,refCountry,Alpha_3,meanIAT,nIAT,seIAT,meanExp,nExp,seExp
0,0.125,lucas-wilchez,Lucas Wilchez,Real Zaragoza,Spain,431136000,177.0,72.0,Attacking Midfielder,1,...,0.50,1,1,GRC,0.326391,712.0,0.000564,0.396000,750.0,0.002696
1,0.000,john-utaka,John Utaka,Montpellier HSC,France,397008000,179.0,82.0,Right Winger,1,...,0.75,2,2,ZMB,0.203375,40.0,0.010875,-0.204082,49.0,0.061504
5,0.125,aaron-hughes,Aaron Hughes,Fulham FC,England,303177600,182.0,71.0,Center Back,1,...,0.00,4,4,LUX,0.325185,127.0,0.003297,0.538462,130.0,0.013752
6,0.125,aleksandar-kolarov,Aleksandar Kolarov,Manchester City,England,497836800,187.0,80.0,Left Fullback,1,...,0.25,4,4,LUX,0.325185,127.0,0.003297,0.538462,130.0,0.013752
7,0.000,alexander-tettey,Alexander Tettey,Norwich City,England,512956800,180.0,68.0,Defensive Midfielder,1,...,1.00,4,4,LUX,0.325185,127.0,0.003297,0.538462,130.0,0.013752


Also, for our classifier to work, we remove all non numerical value by creatting dummy values to replace them.

In [15]:
dummydf = pd.get_dummies(cleandf, prefix=None, prefix_sep='_', dummy_na=False, columns=["position"], sparse=False, drop_first=False)

In [16]:
# count the number of club to check if it feasable to create dummy value for the feature "club"
print("The number of different clubs is",dummydf["club"].unique().size)

The number of different clubs is 97


In [17]:
dummydf = pd.get_dummies(dummydf, prefix=None, prefix_sep='_', dummy_na=False, columns=["club"], sparse=False, drop_first=False)
dummydf = pd.get_dummies(dummydf, prefix=None, prefix_sep='_', dummy_na=False, columns=["leagueCountry"], sparse=False, drop_first=False)

dummydf = dummydf.drop(['playerShort', 'player', 'photoID', 'rater1', 'rater2']  , axis=1)

In [18]:
print("Now we have",dummydf.columns.size, "columns.")

Now we have 134 columns.


In [19]:
dummydf.head()

,meanSkinTone,birthday,height,weight,games,victories,ties,defeats,goals,yellowCards,...,club_Werder Bremen,club_West Bromwich Albion,club_West Ham United,club_Wigan Athletic,club_Wolverhampton Wanderers,club_Évian Thonon Gaillard,leagueCountry_England,leagueCountry_France,leagueCountry_Germany,leagueCountry_Spain
0,0.125,431136000,177.0,72.0,1,0,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.000,397008000,179.0,82.0,1,0,0,1,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
5,0.125,303177600,182.0,71.0,1,0,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
6,0.125,497836800,187.0,80.0,1,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
7,0.000,512956800,180.0,68.0,1,0,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [20]:
dummydf.drop("Alpha_3", axis = 1, inplace = True) 
# val = dummydf["Alpha_3"].drop_duplicates()
# val.apply(lambda c :print(c))

## Classifier

In [34]:
def build_k_indices(y, k_fold, seed):
    """build k indices for k-fold."""
    num_row = len(y)
    interval = int(num_row / k_fold)
    np.random.seed(seed)
    indices = np.random.permutation(num_row)
    k_indices = [indices[k * interval: (k + 1) * interval] for k in range(k_fold)]
    return np.array(k_indices)

In [95]:
import time
from sklearn import linear_model, metrics
from  sklearn.metrics import log_loss

sum_feature_importances = np.zeros(X.shape[1])

def cross_validation(y, x, k_indices, k):
    """return the loss of ridge regression."""
    # get k'th subgroup in test, others in train
    print("kkkkkkkk", k)
    te_indice = k_indices[k]
    tr_indice = k_indices[~(np.arange(k_indices.shape[0]) == k)]
    tr_indice = tr_indice.reshape(-1)
    
    y_te = y[te_indice]
    y_tr = y[tr_indice]
    x_te = x[te_indice]
    x_tr = x[tr_indice]
    
    #Make classifier
    rf = RandomForestClassifier(n_jobs = 4)

    #Train the classifier
    clf = rf.fit(x_tr, y_tr)
    path = rf.feature_importances_
    global sum_feature_importances
    sum_feature_importances = np.add(sum_feature_importances, path)

    # Make prediction for testing data with the classifier
    y_pred = clf.predict(x_te)

    #Compute error (trying with 2 different methods)
    #rf_score1 = log_loss(y_te, y_pred)
    rf_score2 = metrics.accuracy_score(y_te, y_pred)
    
    print(y_pred)

    #print('score: ', rf_score1)
    print('score 2: ', rf_score2)
    

In [44]:
array = pd.DataFrame.as_matrix(dummydf)

#We realize that the dataframe still have some NaN values
# We decided to substitute every NaN with the mean of their column
for i in range(len(dummydf.columns)):
    if (dummydf[dummydf.columns[i]].isnull().values.any()):
        mean = np.mean(dummydf[dummydf.columns[i]])
        dummydf[dummydf.columns[i]].fillna(mean, inplace = True)
dummydf.isnull().values.any()

False

In [79]:
Y = np.asarray(dummydf["meanSkinTone"].values, dtype="|S6")
subDummydf = dummydf.drop("meanSkinTone", axis = 1)
X = subDummydf.as_matrix()

132

In [ ]:
k_fold = 4
k_indices = build_k_indices(Y, k_fold, 1)

for k in range(k_fold):
    cross_validation(Y, X, k_indices, k)

kkkkkkkk 0
[b'0.0' b'0.0' b'0.0' ..., b'0.0' b'0.0' b'0.0']
score 2:  0.996368194382
kkkkkkkk 1
[b'0.0' b'0.0' b'0.0' ..., b'0.0' b'0.125' b'0.0']
score 2:  0.994439802018
kkkkkkkk 2
[b'0.125' b'0.0' b'0.0' ..., b'0.0' b'0.0' b'0.125']
score 2:  0.994761200746
kkkkkkkk 3


In [93]:
sum_feature_importances / 4

array([  1.64026739e-01,   1.24518759e-01,   1.24146538e-01,
         1.06872715e-02,   9.25367149e-03,   6.54104664e-03,
         7.57292768e-03,   6.58647131e-03,   5.94168025e-03,
         6.54326960e-04,   6.63671962e-04,   1.59087597e-02,
         1.10818450e-02,   9.48242348e-03,   9.57672616e-03,
         9.78297375e-03,   1.01735409e-02,   9.46736312e-03,
         9.90360318e-03,   1.15007283e-02,   1.51819445e-02,
         1.48979219e-02,   6.73574853e-03,   1.34411032e-02,
         1.22385877e-02,   1.34710827e-02,   8.21377800e-03,
         7.06290873e-03,   1.21195832e-02,   6.68097329e-03,
         8.17914828e-03,   4.53813093e-03,   5.84470648e-03,
         2.25213644e-03,   1.68184717e-03,   1.56756204e-03,
         1.00503009e-03,   3.59845577e-03,   8.04288452e-06,
         2.54014301e-03,   4.33232721e-03,   4.06685115e-03,
         3.41104205e-03,   3.05246398e-03,   1.64864105e-03,
         1.05540602e-04,   6.86450247e-03,   3.34800177e-03,
         1.49583430e-04,

In [94]:
sum_feature_importances.argsort()

array([ 72,  38,  98,  74, 104,  78,  50, 126,  81, 105,  86,  53, 110,
        45,  77,  48, 117, 102,  96,  87,   9,  10,  71, 115,  36,  61,
       116,  35,  63,  44,  34,  95,  67, 109, 127, 119,  49,  93,  33,
        83,  99,  39,  73, 100,  92,  75,  58, 103,  85,  66,  51, 108,
        94,  43, 123,  57, 124,  80, 101,  47,  90, 125,  82,  42, 111,
        84,  37,  56,  55,  41, 107,  68,  40,  65,  89,  59,  31,  97,
        64, 121,  91,  88, 120, 129,  70,  60, 113, 114,  79,  69, 118,
        76,  32,   8,  52, 122,   5,   7,  29,  22,  46, 112,  27,  62,
       131,   6, 130,  30,  26, 128,   4,  17,  13,  14,  15,  18,  16,
        54,   3,  12,  19,  28,  24,  23,  25,  21,  20, 106,  11,   2,
         1,   0], dtype=int64)